In [1]:
import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.model.RandomForestModel

import org.apache.spark.ml.classification.RandomForestClassifier

import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.101:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1551029661403)
SparkSession available as 'spark'


import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.model.RandomForestModel
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


In [2]:
val session: SparkSession = SparkSession
    .builder()
    .master("local[4]")
    .getOrCreate()

2019-02-24 18:34:27 WARN  SparkSession$Builder:66 - Using an existing SparkSession; some configuration may not take effect.


session: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@517403b2


In [3]:
val dataSetPath = "/home/alcrd/projects/modern scala projects/chapter_01/iris.csv"

dataSetPath: String = /home/alcrd/projects/modern scala projects/chapter_01/iris.csv


In [4]:
 def buildDataFrame(dataSet: String): DataFrame = {
    session.read.format("com.databricks.spark.csv")
      .option("header", true)
      .option("inferSchema", true)
      .load(dataSet)
  }

buildDataFrame: (dataSet: String)org.apache.spark.sql.DataFrame


In [5]:
val df = buildDataFrame(dataSetPath)
df.show(5)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



df: org.apache.spark.sql.DataFrame = [Id: int, SepalLengthCm: double ... 4 more fields]


In [6]:
val summary = df.describe("Id","SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm","Species")
summary.show

+-------+------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|                Id|     SepalLengthCm|       SepalWidthCm|     PetalLengthCm|      PetalWidthCm|       Species|
+-------+------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|               150|                150|               150|               150|           150|
|   mean|              75.5| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          null|
| stddev|43.445367992456916|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          null|
|    min|                 1|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               150|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+----

summary: org.apache.spark.sql.DataFrame = [summary: string, Id: string ... 5 more fields]


In [7]:
val assembler = new VectorAssembler()
  .setInputCols(Array("SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"))
  .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_a76f54aff412


In [8]:
val df_transformed = assembler.transform(df)
df_transformed.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|         features|
+---+-------------+------------+-------------+------------+-----------+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+---+-------------+------------+-------------+------------+-----------+-----------------+
only showing top 5 rows



df_transformed: org.apache.spark.sql.DataFrame = [Id: int, SepalLengthCm: double ... 5 more fields]


In [9]:
val indexer = new StringIndexer()
  .setInputCol("Species")
  .setOutputCol("Label")
  .fit(df_transformed)
  
val df_in = indexer.transform(df_transformed)
df_in.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------------+-----+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|         features|Label|
+---+-------------+------------+-------------+------------+-----------+-----------------+-----+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|  0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|  0.0|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|  0.0|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|  0.0|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|  0.0|
+---+-------------+------------+-------------+------------+-----------+-----------------+-----+
only showing top 5 rows



indexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_1c578cbd7012
df_in: org.apache.spark.sql.DataFrame = [Id: int, SepalLengthCm: double ... 6 more fields]


In [10]:
val df_subset = df_in.select("label", "features")
df_subset.show

+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[5.1,3.5,1.4,0.2]|
|  0.0|[4.9,3.0,1.4,0.2]|
|  0.0|[4.7,3.2,1.3,0.2]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[5.0,3.6,1.4,0.2]|
|  0.0|[5.4,3.9,1.7,0.4]|
|  0.0|[4.6,3.4,1.4,0.3]|
|  0.0|[5.0,3.4,1.5,0.2]|
|  0.0|[4.4,2.9,1.4,0.2]|
|  0.0|[4.9,3.1,1.5,0.1]|
|  0.0|[5.4,3.7,1.5,0.2]|
|  0.0|[4.8,3.4,1.6,0.2]|
|  0.0|[4.8,3.0,1.4,0.1]|
|  0.0|[4.3,3.0,1.1,0.1]|
|  0.0|[5.8,4.0,1.2,0.2]|
|  0.0|[5.7,4.4,1.5,0.4]|
|  0.0|[5.4,3.9,1.3,0.4]|
|  0.0|[5.1,3.5,1.4,0.3]|
|  0.0|[5.7,3.8,1.7,0.3]|
|  0.0|[5.1,3.8,1.5,0.3]|
+-----+-----------------+
only showing top 20 rows



df_subset: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [11]:
val dfs = df_subset.randomSplit(Array(.80, .20))
val (df_train, df_test) = (dfs(0), dfs(1))

dfs: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array([label: double, features: vector], [label: double, features: vector])
df_train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
df_test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


In [12]:
val rf = new RandomForestClassifier()
    .setLabelCol("label")
    .setFeaturesCol("features")
    .setNumTrees(10)

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_a1ac7aad113a


In [13]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_8e42774e7516


In [14]:
val gridSearch = new ParamGridBuilder()
    .addGrid(rf.maxDepth, Array(4, 5, 6, 10))
    .addGrid(rf.impurity, Array("gini", "entropy"))
    .build()

gridSearch: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_a1ac7aad113a-impurity: gini,
	rfc_a1ac7aad113a-maxDepth: 4
}, {
	rfc_a1ac7aad113a-impurity: entropy,
	rfc_a1ac7aad113a-maxDepth: 4
}, {
	rfc_a1ac7aad113a-impurity: gini,
	rfc_a1ac7aad113a-maxDepth: 5
}, {
	rfc_a1ac7aad113a-impurity: entropy,
	rfc_a1ac7aad113a-maxDepth: 5
}, {
	rfc_a1ac7aad113a-impurity: gini,
	rfc_a1ac7aad113a-maxDepth: 6
}, {
	rfc_a1ac7aad113a-impurity: entropy,
	rfc_a1ac7aad113a-maxDepth: 6
}, {
	rfc_a1ac7aad113a-impurity: gini,
	rfc_a1ac7aad113a-maxDepth: 10
}, {
	rfc_a1ac7aad113a-impurity: entropy,
	rfc_a1ac7aad113a-maxDepth: 10
})


In [15]:
val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(rf)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(gridSearch)
  .setTrainRatio(0.8)

trainValidationSplit: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_95c8d79b942e


In [16]:
val model = trainValidationSplit.fit(df_train)

model: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_95c8d79b942e


In [17]:
val pred = model.transform(df_test)

pred: org.apache.spark.sql.DataFrame = [label: double, features: vector ... 3 more fields]


In [18]:
pred.show()

+-----+-----------------+--------------------+--------------------+----------+
|label|         features|       rawPrediction|         probability|prediction|
+-----+-----------------+--------------------+--------------------+----------+
|  0.0|[4.4,2.9,1.4,0.2]|      [10.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|  0.0|[4.6,3.4,1.4,0.3]|      [10.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|  0.0|[4.8,3.1,1.6,0.2]|      [10.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|  0.0|[4.8,3.4,1.6,0.2]|      [10.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|  0.0|[4.8,3.4,1.9,0.2]|[8.0,1.9555555555...|[0.8,0.1955555555...|       0.0|
|  0.0|[4.9,3.0,1.4,0.2]|      [10.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|  0.0|[5.0,3.2,1.2,0.2]|      [10.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|  0.0|[5.1,3.3,1.7,0.5]|[9.0,0.9555555555...|[0.9,0.0955555555...|       0.0|
|  0.0|[5.1,3.4,1.5,0.2]|      [10.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|  0.0|[5.1,3.5,1.4,0.3]|      [10.0,0.0,0.0]|      

In [20]:
val accuracy = evaluator.evaluate(pred)
println(s"Test Error = ${(1.0 - accuracy)}")

Test Error = 0.04761904761904767


accuracy: Double = 0.9523809523809523
